In [32]:
import tensorflow as tf
import numpy as np


class ELM(object):
  def __init__(self, sess, batch_size, input_len, hidden_num, output_len, w, b, beta):

    self._sess = sess 
    self._batch_size = batch_size
    self._input_len = input_len
    self._hidden_num = hidden_num
    self._output_len = output_len 

    # for train
    self._x0 = tf.placeholder(tf.float32, [self._batch_size, self._input_len])
    self._t0 = tf.placeholder(tf.float32, [self._batch_size, self._output_len])

    # for test
    self._x1 = tf.placeholder(tf.float32, [None, self._input_len])
    self._t1 = tf.placeholder(tf.float32, [None, self._output_len])

    self._W = w
    self._b = b
    self._beta = beta
    #self._var_list = [self._W, self._b, self._beta]

    self.H0 = tf.matmul(self._x0, self._W) + self._b # N x L
    self.H0_T = tf.transpose(self.H0)

    self.H1 = tf.matmul(self._x1, self._W) + self._b # N x L
    self.H1_T = tf.transpose(self.H1)

    # beta analytic solution : self._beta_s (K x O)
    if self._input_len < self._hidden_num: # L < K
      identity = tf.constant(np.identity(self._hidden_num), dtype=tf.float32)
      self._beta_s = tf.matmul(tf.matmul(tf.matrix_inverse(
        tf.matmul(self.H0_T, self.H0) + identity), 
        self.H0_T), self._t0)
      # _beta_s = (H_T*H + I/om)^(-1)*H_T*T
    else:
      identity = tf.constant(np.identity(self._batch_size), dtype=tf.float32)
      self._beta_s = tf.matmul(tf.matmul(self.H0_T, tf.matrix_inverse(
        tf.matmul(self.H0, self.H0_T)+identity)), self._t0)
      # _beta_s = H_T*(H*H_T + I/om)^(-1)*T
    
    self._assign_beta = self._beta.assign(self._beta_s)
    self._fx0 = tf.matmul(self.H0, self._beta)
    self._fx1 = tf.matmul(self.H1, self._beta)

    self._cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self._fx0, labels=self._t0))

    # for the mnist test
    self._correct_prediction = tf.equal(tf.argmax(self._fx1,1), tf.argmax(self._t1,1))
    self._accuracy = tf.reduce_mean(tf.cast(self._correct_prediction, tf.float32))

  def feed(self, x, t):
      
    self._sess.run(self._assign_beta, {self._x0:x, self._t0:t})

  def test(self, x, t=None):
    if t is not None :
      print("Accuracy: {:.9f}".format(self._sess.run(self._accuracy, {self._x1:x, self._t1:t})))
    else :
      return self._sess.run(self._fx1, {self._x1:x})

In [35]:
#ELM TEST

#from model import ELM
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Basic tf setting
tf.set_random_seed(2016)

batch_size = 5000
hidden_num = 100
input_len = 784
output_len = 10

_W = tf.Variable(tf.random_normal([input_len, hidden_num]),trainable=False, dtype=tf.float32)
_b = tf.Variable(tf.random_normal([hidden_num]),trainable=False, dtype=tf.float32)
_beta = tf.Variable( tf.zeros([hidden_num, output_len]),trainable=False, dtype=tf.float32)

init = tf.global_variables_initializer()


with tf.Session() as sess:

    # Get data
    mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
    sess.run(init)
    # Construct ELM
    
    print("batch_size : {}".format(batch_size))
    print("hidden_num : {}".format(hidden_num))
    elm = ELM(sess, batch_size, input_len, hidden_num, output_len,_W,_b,_beta)

    # one-step feed-forward training
    print("Accuracy on test before")
    elm.test(mnist.test.images, mnist.test.labels)    
    train_x, train_y = mnist.train.next_batch(batch_size)
    elm.feed(train_x, train_y)

    # testing
    # Accuracy on test after
    print("Accuracy on test after")
    elm.test(mnist.test.images, mnist.test.labels)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
batch_size : 5000
hidden_num : 100
Accuracy on test before
(10000, 784)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Accuracy: 0.097999997
Accuracy on test after
Accuracy: 0.826399982


In [ ]:
"""
150 nodes
Accuracy: 0.836199999
100 nodes
Accuracy: 0.826399982
"""